# Set up environment

In [1]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

In [2]:
pip install -r D:\shuzhi\monai_replicode\tutorials-main\tutorials-main\requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -r https://raw.githubusercontent.com/Project-MONAI/MONAI/dev/requirements-dev.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='raw.githubusercontent.com', port=443): Max retries exceeded with url: /Project-MONAI/MONAI/dev/requirements-dev.txt (Caused by NewConnectionError('<pip._vendor.urllib3.connection.HTTPSConnection object at 0x000002DF71F44EB0>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed'))



# Set up imports

In [5]:
from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    Invertd,
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob

print_config()

MONAI version: 1.5.dev2516
Numpy version: 2.2.5
Pytorch version: 2.6.0+cpu
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 875aa3d53281b83a205442db83ac41ddf817ef9a
MONAI __file__: d:\shuzhi\monai_replicode\monai\lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.4.3
Nibabel version: 5.3.2
scikit-image version: 0.25.2
scipy version: 1.15.2
Pillow version: 11.2.1
Tensorboard version: 2.19.0
gdown version: 5.2.0
TorchVision version: 0.21.0+cpu
tqdm version: 4.67.1
lmdb version: 1.6.2
psutil version: 7.0.0
pandas version: 2.2.3
einops version: 0.8.1
transformers version: 4.40.2
mlflow version: 2.21.3
pynrrd version: 1.1.3
clearml version: 2.0.0rc0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



# Set up data directory

In [6]:
# 直接指定路径（无需环境变量）
# Set specific path
directory = r"D:\shuzhi\monai_replicode\monai_datasets"  # 使用原始字符串避免转义
os.makedirs(directory, exist_ok=True)
root_dir = directory
print("数据目录路径:", root_dir)

数据目录路径: D:\shuzhi\monai_replicode\monai_datasets


# Download dataset

https://msd-for-monai.s3-us-west-2.amazonaws.com/Task09_Spleen.tar
Run 'tar -xf .\Task09_Spleen.tar' in Terminal to extract

In [7]:
compressed_file = os.path.join(root_dir, "Task09_Spleen.tar")
data_dir = os.path.join(root_dir, "Task09_Spleen")

In [8]:
print("压缩包路径：", compressed_file)  # 应为 D:\...\MedNIST.tar.gz
print("数据目录路径：", data_dir)       # 应为 D:\...\MedNIST

# 打印绝对路径
print("root_dir path:", os.path.abspath(root_dir))
print("data_dir path:", os.path.abspath(data_dir))

压缩包路径： D:\shuzhi\monai_replicode\monai_datasets\Task09_Spleen.tar
数据目录路径： D:\shuzhi\monai_replicode\monai_datasets\Task09_Spleen
root_dir path: D:\shuzhi\monai_replicode\monai_datasets
data_dir path: D:\shuzhi\monai_replicode\monai_datasets\Task09_Spleen


# Set MSD Spleen dataset path

In [9]:
# ================== 数据路径处理（不属于 Data Transform） ==================
# 获取训练图像和标签文件路径列表（通过glob匹配.nii.gz文件）

train_images = sorted(glob.glob(os.path.join(data_dir, "imagesTr", "*.nii.gz")))
train_labels = sorted(glob.glob(os.path.join(data_dir, "labelsTr", "*.nii.gz")))

# 将图像和标签路径组合成字典列表（MONAI数据集的标准输入格式）
data_dicts =[
    {"image": image_name, "label": label_name} for image_name, label_name in zip(train_images, train_labels)
]

# 划分训练集和验证集（取最后9个样本作为验证集）
train_files, val_files = data_dicts[:-9], data_dicts[-9:]

# Set deterministic training for reproducibility

In [10]:
set_determinism(seed=0)

# Setup transforms for training and validation

In [11]:
# ================== 核心 Data Transform 部分 ==================
# 定义训练集的数据增强流水线
# define the data transforms
train_transforms = Compose(
    [
        # 1. 加载NIfTI文件
        LoadImaged(keys=["image", "label"]),

        # 2. 确保数据有通道维度（形状从 (H,W,D) 变为 (C,H,W,D)）
        EnsureChannelFirstd(keys=["image", "label"]),

        # 3. 统一图像方向（例如将LAS坐标系转为RAS）
        Orientationd(keys=["image", "label"], axcodes="RAS"),

        # 4. 重采样到统一分辨率（1.5x1.5x2.0毫米）
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.5, 1.5, 2.0), # XYZ轴的目标间距
            mode=("bilinear", "nearest"), # 图像双线性插值，标签最近邻
        ),

        # 5. 调整CT值范围并归一化到[0,1]
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-57, # 最低CT值（过滤无关组织）
            a_max=164, # 最高CT值（聚焦脾脏）
            b_min=0.0,
            b_max=1.0, 
            clip=True    # 超出范围的像素截断到[a_min, a_max]
        ),

        # 6. 裁剪非零区域（去除图像边缘的无效背景）
        CropForegroundd(keys=["image", "label"], source_key="image"),

        # 7. 随机裁剪带标签的3D块（数据增强关键步骤）
        # randomly crop out patch samples from
        # big image based on pos / neg ratio
        # the image centers of negative samples
        # must be in valid image area
        RandCropByPosNegLabeld(
            keys=["image", "label"],
            label_key="label", # 根据标签生成正负样本
            spatial_size=(96, 96, 96), # 裁剪块尺寸
            pos=1, # 正样本（包含目标）的比例
            neg=1, # 负样本（不包含目标）的比例
            num_samples=4, # 每个输入样本生成4个块
            image_key="image",
            image_threshold=0, # 像素值阈值定义有效区域
                ),
                # user can also add other random transforms
                #                 RandAffined(
                #                     keys=['image', 'label'],
                #                     mode=('bilinear', 'nearest'),
                #                     prob=1.0,
                #                     spatial_size=(96, 96, 96),
                #                     rotate_range=(0, 0, np.pi/15),
                #                     scale_range=(0.1, 0.1, 0.1)),
    ]
)

monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.


In [12]:
# 定义验证集的预处理流水线（无随机增强）
val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.5, 1.5, 2.0),
            mode=("bilinear", "nearest"),
                ),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-57,
            a_max=164,
            b_min=0.0,
            b_max=1.0,
            clip=True,
                ),
        CropForegroundd(keys=["image", "label"], source_key="image"),
    ]
)

# Define CacheDataset and DataLoader for training and validation

In [13]:
# 创建验证集 DataLoader（应用预处理）
val_ds = CacheDataset(data=val_files, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=1, num_workers=2)  # 批量大小设为1方便可视化



# ============================================================

# train_ds = CacheDataset(data=train_files, transform=train_transforms, cache_rate=1.0, num_workers=4)
# # train_ds = Dataset(data=train_files, transform=train_transforms)

# # use batch_size=2 to load images and use RandCropByPosNegLabeld
# # to generate 2 x 4 images for network training
# train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=4)

# val_ds = CacheDataset(data=val_files, transform=val_transforms, cache_rate=1.0, num_workers=4)
# # val_ds = Dataset(data=val_files, transform=val_transforms)
# val_loader = DataLoader(val_ds, batch_size=1, num_workers=4)

Loading dataset: 100%|██████████| 9/9 [00:46<00:00,  5.15s/it]


# Load the model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device)

# 加载本地预训练权重
model_path = r"D:\shuzhi\monai_replicode\monai_test\model.pt"  # 原始路径
model.load_state_dict(torch.load(model_path, map_location=device))  # 自动适应CPU/GPU

model.eval()

UNet(
  (model): Sequential(
    (0): ResidualUnit(
      (conv): Sequential(
        (unit0): Convolution(
          (conv): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
          (adn): ADN(
            (N): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
        (unit1): Convolution(
          (conv): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          (adn): ADN(
            (N): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
      )
      (residual): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    )
    (1): SkipConnection(
      (submodule): Sequential(
        (0): ResidualUnit(
          (conv): Sequential(


: 

# Check best model output with the input image and label

In [ ]:
with torch.no_grad():
    for i, val_data in enumerate(val_loader):
        # 滑动窗口推理（适合大尺寸图像）
        val_outputs = sliding_window_inference(
            inputs=val_data["image"].to(device),
            roi_size=(96, 96, 96),  # Match training crop size
            sw_batch_size=4,        # Number of sliding window batches
            predictor=model,
            overlap=0.5             # Overlap ratio to reduce boundary artifacts
        )
        
        # 显示中间切片（第80层）
        plt.figure(f"Validation Sample {i}", (18, 6))
        plt.subplot(1, 3, 1)
        plt.title("Original CT Image")  # 原始CT图像 → Original CT Image
        plt.imshow(val_data["image"][0, 0, :, :, 80], cmap="gray")
        
        plt.subplot(1, 3, 2)
        plt.title("Ground Truth (Spleen)")  # 医生标注（脾脏） → Ground Truth (Spleen)
        plt.imshow(val_data["label"][0, 0, :, :, 80])
        
        plt.subplot(1, 3, 3)
        plt.title("Model Prediction")  # 模型预测 → Model Prediction
        pred_mask = torch.argmax(val_outputs, dim=1)
        plt.imshow(pred_mask[0, :, :, 80].cpu().numpy())
        
        plt.show()
        if i >= 2:  # Only show first 3 samples
            break

# Evaluation on original image spacings

In [19]:
val_org_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image"], axcodes="RAS"),
        Spacingd(keys=["image"], pixdim=(1.5, 1.5, 2.0), mode="bilinear"),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-57,
            a_max=164,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["image"], source_key="image"),
    ]
)

val_org_ds = Dataset(data=val_files, transform=val_org_transforms)
val_org_loader = DataLoader(val_org_ds, batch_size=1, num_workers=4)

post_transforms = Compose(
    [
        Invertd(
            keys="pred",
            transform=val_org_transforms,
            orig_keys="image",
            meta_keys="pred_meta_dict",
            orig_meta_keys="image_meta_dict",
            meta_key_postfix="meta_dict",
            nearest_interp=False,
            to_tensor=True,
            device="cpu",
        ),
        AsDiscreted(keys="pred", argmax=True, to_onehot=2),
        AsDiscreted(keys="label", to_onehot=2),
    ]
)

monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.


In [ ]:
model_path = r"D:\shuzhi\monai_replicode\monai_test\model.pt"  # 原始路径
model.load_state_dict(torch.load(model_path, map_location=device))  # 自动适应CPU/GPU

model.eval()

with torch.no_grad():
    for val_data in val_org_loader:
        val_inputs = val_data["image"].to(device)
        roi_size = (160, 160, 160)
        sw_batch_size = 4
        val_data["pred"] = sliding_window_inference(val_inputs, roi_size, sw_batch_size, model)
        val_data = [post_transforms(i) for i in decollate_batch(val_data)]
        val_outputs, val_labels = from_engine(["pred", "label"])(val_data)
        # compute metric for current iteration
        dice_metric(y_pred=val_outputs, y=val_labels)

#     # aggregate the final mean dice result
#     metric_org = dice_metric.aggregate().item()
#     # reset the status for next validation round
#     dice_metric.reset()

# print("Metric on original image spacing: ", metric_org)